In [1]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import matplotlib.colors as mcolors

Befor runing this code: 
1) creaea an empty folder (input_folder) in your desired location and writhe down the path below! 
2) Change the corresponding Q CODE from wikidata for a given Genus in the code below!

That's all

DEFINE THE FOLLOWING PARAMETERS

In [2]:
#Paths
# Input folder containing CSV files
input_folder = "//FARMA-AD2.farma.unige.ch/Pharmacie/Recherche/COMMON FASIE-FATHO/Luis/Side Projects/article Lisa/"

#Swertia chirayita (Q21318003)
#Swertia (Q163970)
#Gentianaceae (Q157216)
species_name = 'Combretum micranthum' #'your_species_name_here'  # Replace 'your_species_name_here' with the actual species name
qcode = "Q1245859"  # Replace with the actual Q-code for the genus

#lotus databases
LOTUSDB = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'
LOTUSDB_rc = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_rc.csv'

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:

#functions to colors palets, etc
def fetch_species_from_qcode(qcode):
    """
    Fetches all species under a given genus using the Wikidata SPARQL endpoint.

    Parameters:
    genus_qcode (str): The Wikidata Q-code for the genus.

    Returns:
    pd.DataFrame: A DataFrame containing the species and their corresponding Q-codes.
    """

    # SPARQL query to fetch species under a given genus
    query = """
    SELECT ?species ?speciesLabel WHERE {
      ?species wdt:P171* wd:%s .
      ?species wdt:P105 wd:Q7432.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """ % qcode

    # URL for the Wikidata SPARQL endpoint
    url = "https://query.wikidata.org/sparql"

    # Request headers
    headers = {
        "User-Agent": "Wikidata Species Fetcher/0.1 (https://www.wikidata.org/wiki/Wikidata:Data_access)"
    }

    # Perform the request
    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code != 200:
        raise Exception("Failed to fetch data: HTTP status code {}".format(response.status_code))

    # Parse the JSON response
    data = response.json()

    # Extract results
    species = []
    for item in data['results']['bindings']:
        species.append({
            'wikidata_Qcode_species': item['species']['value'].split('/')[-1],
            'Species': item['speciesLabel']['value']
        })

    return pd.DataFrame(species)

# Define base colors for each Pathway
pathway_shades= {
    'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
    'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
    'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
    'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
    'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
    'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
    'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
}

# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Terpenoids': '#618264',  # Green start and lighter green end
    'Alkaloids': '#305F72',   # Blue start and lighter blue end
    'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
    'Polyketides': '#EF4B4B',  # Red start and lighter purple end
    'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
    'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
    'Carbohydrates': '#65451F' # Brown start and lighter purple end
}

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    base_color, end_color = pathway_shades.get(pathway, ('gray', 'lightgray'))
    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'

FIRST RECOVER THE SPECIES (and Q codes) BELONGING TO A PARTICULAR GENUS 

In [5]:
# Usage
species_df = fetch_species_from_qcode(qcode)
species_df.head()  # Display the first few rows

# Filename for the CSV file
file_name = "species_list.csv"

# Full path for the CSV file
full_file_path = f"{input_folder}/{file_name}"

# Save the DataFrame to a CSV file in the specified folder
species_df.to_csv(full_file_path, index=False)

NOW, LETS USE THE QCODES TO RECOVER ALL THE COMPOUNDS (WITH REFERENCES) PRESENT IN EACH QCODE AND SAVE THEM AS INDIVIDUAL CSV FILES FROM LOTUS_DB FROZEN METADATA

In [6]:
# Load the LOTUSDB CSV
lotusdb_df = pd.read_csv(LOTUSDB, low_memory=False)

# Iterate through all CSV files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_folder, filename)

        # Load the CSV file with Q codes
        df_species = pd.read_csv(input_file)

        # Iterate through the Q codes in the species CSV
        for q_code in df_species['wikidata_Qcode_species']:  # Ensure this column name matches your species CSV
            # Skip processing if Q code is 'Not Found'
            if q_code == 'Not Found':
                continue

            # Filter LOTUSDB data for the current Q code
            filtered_lotusdb = lotusdb_df[lotusdb_df['wikidata_Qcode'] == q_code]  # Ensure this column name matches your LOTUSDB CSV

            # Group and aggregate the data
            grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
                # Add all the aggregation rules here
                # Example:
                "structure_wikidata": "first",
                "structure_inchi": "first",
                "structure_smiles": "first",
                "structure_molecular_formula": "first",
                "structure_exact_mass": "first",
                "structure_xlogp": "first",
                "structure_smiles_2D": "first",
                "structure_cid": "first",
                "structure_nameIupac": "first",
                "structure_nameTraditional": "first",
                "structure_taxonomy_npclassifier_01pathway": "first",
                "structure_taxonomy_npclassifier_02superclass": "first",
                "structure_taxonomy_npclassifier_03class": "first",
                "organism_wikidata": "first",
                "organism_taxonomy_gbifid": "first",
                "organism_taxonomy_ncbiid": "first",
                "organism_taxonomy_ottid": "first",
                "organism_taxonomy_01domain": "first",
                "organism_taxonomy_02kingdom": "first",
                "organism_taxonomy_03phylum": "first",
                "organism_taxonomy_04class": "first",
                "organism_taxonomy_05order": "first",
                "organism_taxonomy_06family": "first",
                "organism_taxonomy_07tribe": "first",
                "organism_taxonomy_08genus": "first",
                "organism_taxonomy_09species": "first",
                "organism_taxonomy_10varietas": "first",
                "reference_wikidata": lambda x: "|".join(map(str, x)),
                "reference_doi": lambda x: "|".join(map(str, x))
            }).reset_index()
            
            # Create 'chemical_superclass' and 'chemical_class' columns
            grouped_df['chemical_superclass'] = grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' + grouped_df['structure_taxonomy_npclassifier_02superclass']
            grouped_df['chemical_class'] = grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' + grouped_df['structure_taxonomy_npclassifier_03class']
            
            # Save the grouped data as a TSV file with the Q code as the filename
            output_subfolder = os.path.join(output_folder, 'species_data')
            #if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder, exist_ok=True)

            output_filename = os.path.join(output_subfolder, f"{q_code}.tsv")
            grouped_df.to_csv(output_filename, index=False, sep='\t')

            print(f"Saved grouped data for Q code {q_code} to {output_filename}")


Saved grouped data for Q code Q1245859 to //FARMA-AD2.farma.unige.ch/Pharmacie/Recherche/COMMON FASIE-FATHO/Luis/Side Projects/article Lisa/output_data/species_data\Q1245859.tsv


RECOVER THE FRECUENCE BY CHEMICAL CLASS AND ADDED TO THE GENERAL TABLE

In [7]:
# Step 1: Initialize variables to store data
total_compounds = {}
chemical_classes = {}
chemical_superclasses = {}
references = {}
hyperlinks = {}

# Step 2: Iterate through .tsv files in the folder

species_data_folder = os.path.join(output_folder, 'species_data')
for filename in os.listdir(species_data_folder):
    if filename.endswith(".tsv"):
        qcode = filename.split(".")[0]  # Extract Qcode from the filename

        # Load the .tsv file into a DataFrame
        df_compounds = pd.read_csv(os.path.join(species_data_folder, filename), sep='\t')

        # Step 3: Calculate frequencies of chemical classes for each Qcode (excluding 'not classified')
        filtered_classes = df_compounds[df_compounds['structure_taxonomy_npclassifier_03class'] != 'Not Classified']
        grouped = filtered_classes['structure_taxonomy_npclassifier_03class'].value_counts().reset_index()
        frequencies = grouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_03class']}", axis=1)
        chemical_classes[qcode] = "|".join(frequencies)

        # Step 4: Calculate frequencies of chemical superclasses for each Qcode (excluding 'not classified')
        filtered_sclasses = df_compounds[df_compounds['structure_taxonomy_npclassifier_02superclass'] != 'Not Classified']
        sgrouped = filtered_sclasses['structure_taxonomy_npclassifier_02superclass'].value_counts().reset_index()
        frequencies = sgrouped.apply(lambda row: f"{row.name} {row['structure_taxonomy_npclassifier_02superclass']}", axis=1)
        chemical_superclasses[qcode] = "|".join(frequencies)
        chemical_classes[qcode] = "|".join(frequencies)
        
        # Step 5: Combine references and hyperlinks
        #references[qcode] = "|".join(df_compounds['reference_wikidata'].dropna())
        #hyperlinks[qcode] = "|".join(df_compounds['reference_doi'].dropna())

        # Step 6: Calculate the total number of reported compounds
        #total_compounds[qcode] = len(df_compounds)

# Step 7: Load the general info .csv into a DataFrame
csv_file = None
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        csv_file = os.path.join(input_folder, filename)
        break  # Stop searching after finding the first CSV file

if csv_file is not None:
    # Load the CSV file into the df_general_info DataFrame
    df_general_info = pd.read_csv(csv_file)

# Step 7: Create new columns for combined information
df_general_info['predicted_class'] = df_general_info['wikidata_Qcode_species'].map(chemical_classes)
df_general_info['predicted_superclass'] = df_general_info['wikidata_Qcode_species'].map(chemical_superclasses)
#df_general_info['references'] = df_general_info['wikidata_Qcode_species'].map(references)
#df_general_info['referenceLabel'] = df_general_info['wikidata_Qcode_species'].map(hyperlinks)

# Step 7: Save the updated DataFrame to the .csv file

# Replace values matching the pattern with an empty cell
df_general_info['predicted_class'] = df_general_info['predicted_class'].replace('index|predicted_class', '')
df_general_info['predicted_superclass'] = df_general_info['predicted_superclass'].replace('index|predicted_superclass', '')

#add reported compounds to each Q code
LotusDB_rc = pd.read_csv(LOTUSDB_rc)

spDB = LotusDB_rc[['wikidata_Qcode', 'Reported_comp_Species']].drop_duplicates()
gDB = LotusDB_rc[['organism_taxonomy_08genus', 'Reported_comp_Genus']].drop_duplicates()
fDB = LotusDB_rc[['organism_taxonomy_06family', 'Reported_comp_Family']].drop_duplicates()

df = pd.merge(df_general_info, spDB,
            how= 'left', left_on='wikidata_Qcode_species', right_on='wikidata_Qcode')#.drop_duplicates(subset=['CODE'])
df.drop('wikidata_Qcode', axis=1, inplace=True)

#df = pd.merge(df, gDB,
#            how= 'left', left_on='genus', right_on='organism_taxonomy_08genus')#.drop_duplicates(subset=['full_species'])
#df.drop('organism_taxonomy_08genus', axis=1, inplace=True)

#df = pd.merge(df, fDB,
#            how= 'left', left_on='family', right_on='organism_taxonomy_06family')#.drop_duplicates(subset=['full_species'])
#df.drop('organism_taxonomy_06family', axis=1, inplace=True)

output_csv_file = output_folder + 'Full_results.csv'
df.to_csv(output_csv_file, index=False, sep =',')

# BARPLOT CHEMICAL CLASS - SPECIES 

In [8]:
import pandas as pd
import plotly.express as px

# Step 1: Read data from all .tsv files in the output_folder
all_data = pd.concat([pd.read_csv(os.path.join(output_folder, 'species_data', filename), sep='\t') for filename in os.listdir(os.path.join(output_folder, 'species_data')) if filename.endswith(".tsv")])

# Step 2: Rename the "organism_taxonomy_09species" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[~all_data['chemical_class'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

# Apply the function to each row
all_data[['Pathway', 'chemical_class']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['chemical_class'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence for a specific species

agg_data = all_data[all_data['species'] == species_name].groupby('chemical_class').size().reset_index(name='recurrence')

# Get unique chemical superclasses and sort them alphabetically
unique_superclasses = sorted(agg_data['chemical_class'].unique())

# Create the bar plot
fig = px.bar(agg_data, x='chemical_class', y='recurrence',
             title=f'Barplot of Predicted Superclasses Occurrence for {species_name}',
             labels={'recurrence': 'Recurrence'},
             color='chemical_class',
             color_discrete_map=color_map,
             category_orders={'chemical_class': unique_superclasses},
             orientation='v')  # Vertical bar plot

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1200, height=800)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_class_barplot_{species_name}.html')

# Show the figure
fig.show()

# Create the sunburst plot
fig = px.sunburst(agg_data, path=['chemical_class'], values='recurrence',
                  title=f'Sunburst Plot of Chemical Superclasses Occurrence for {species_name}',
                  color='chemical_class',
                  color_discrete_map=color_map)

# Modify the size of the figure
fig.update_layout(width=800, height=800)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_class_sunburst_{species_name}.html')

# Show the figure
fig.show()


# BARPLOT CHEMICAL_SUPERCLASS - SPECIES 

In [9]:
import pandas as pd
import plotly.express as px

# Step 1: Read data from all .tsv files in the output_folder
all_data = pd.concat([pd.read_csv(os.path.join(output_folder, 'species_data', filename), sep='\t') for filename in os.listdir(os.path.join(output_folder, 'species_data')) if filename.endswith(".tsv")])

# Step 2: Rename the "organism_taxonomy_09species" column to "species"
all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[~all_data['chemical_superclass'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

# Apply the function to each row
all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

# Step 4: Process data for color mapping
color_map = {}
for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
    shades = generate_shades(pathway, len(superclasses))
    for superclass, shade in zip(superclasses, shades):
        color_map[f"{pathway}-{superclass}"] = shade

# Step 5: Group and aggregate data to calculate recurrence for a specific species

agg_data = all_data[all_data['species'] == species_name].groupby('chemical_superclass').size().reset_index(name='recurrence')

# Get unique chemical superclasses and sort them alphabetically
unique_superclasses = sorted(agg_data['chemical_superclass'].unique())

# Create the bar plot
fig = px.bar(agg_data, x='chemical_superclass', y='recurrence',
             title=f'Barplot of Predicted Superclasses Occurrence for {species_name}',
             labels={'recurrence': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_map=color_map,
             category_orders={'chemical_superclass': unique_superclasses},
             orientation='v')  # Vertical bar plot

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1200, height=800)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_barplot_{species_name}.html')

# Show the figure
fig.show()

# Create the sunburst plot
fig = px.sunburst(agg_data, path=['chemical_superclass'], values='recurrence',
                  title=f'Sunburst Plot of Chemical Superclasses Occurrence for {species_name}',
                  color='chemical_superclass',
                  color_discrete_map=color_map)

# Modify the size of the figure
fig.update_layout(width=800, height=800)

# Save the figure as an HTML file
fig.write_html(f'{output_folder}Wikidata_superclass_sunburst_{species_name}.html')

# Show the figure
fig.show()
